In [123]:
import re
import pandas as pd
from datetime import datetime, time
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
from copy import copy
from pathlib import Path
import shutil
import os

In [124]:
IDIOMAS_VALIDOS = ["INGLÉS", "PORTUGUÉS", "ITALIANO", "QUECHUA"]

IDIOMA_ABBR = {
    "INGLÉS": "ING",
    "PORTUGUÉS": "PORT",
    "ITALIANO": "ITA",
    "QUECHUA": "QUE"
}
NIVEL_ABBR = {
    "Básico": "B",
    "Intermedio": "I",
    "Avanzado": "A"
}
MODALIDAD_ABBR = {
    "regular": "REG",
    "intensivo": "INT",
    "súperintensivo": "SINT",
    "superintensivo": "SINT",
    "repaso": "REP"
}
DIA_COD = {0: "L", 1: "M", 2: "X", 3: "J", 4: "V", 5: "S", 6: "D"}

DIAS_VALIDOS = ["LUNES", "MARTES", "MIÉRCOLES", "JUEVES", "VIERNES", "SÁBADOS", "DOMINGOS"]
CAMPOS_FECHA = ["F. Inicio", "F. Fin", "Parcial", "Final", "Subida de notas"]
COLUMNAS_ENTERAS = [
    "Ciclo", "N° Inscritos", "N° Esperado",
    "N° Aprobados", "N° Desaprobados", "N° No asistio (tiene 0)"
]
COLUMNAS_FINALES = [
    "CODIGO", "Nivel", "Ciclo", "MODALIDAD", "DOCENTE", "IDIOMA", "DÍAS DETECTADOS",
    "HORARIO DETALLADO", "F. Inicio", "F. Fin", "Parcial", "Final", "Subida de notas",
    "N° Inscritos", "N° Esperado", "N° Aprobados", "N° Desaprobados",
    "N° No asistio (tiene 0)", "Destalle del curso"
]


In [125]:
def clean_df_mes_idioma(excel_path, mes, idioma_input):
    import pandas as pd
    import re
    from datetime import datetime, time

    # Leer hoja
    df = pd.read_excel(excel_path, sheet_name=mes, skiprows=1)
    matricula_idx = df[df.iloc[:, 0].astype(str).str.upper().str.contains("MATRÍCULA")].index
    if not matricula_idx.empty:
        df = df.loc[:matricula_idx[0] - 1]

    # Etiquetar idioma
    idiomas_validos = ["INGLÉS", "PORTUGUÉS", "ITALIANO", "QUECHUA"]
    df["IDIOMA"] = None
    idioma_actual = "INGLÉS"
    for i, row in df.iterrows():
        cod_val = str(row.get("CODIGO", "")).strip().upper()
        ciclo_val = str(row.get("CICLO", "")).strip().upper()
        for idioma in idiomas_validos[1:]:
            if idioma in cod_val or idioma in ciclo_val:
                idioma_actual = idioma
                break
        df.at[i, "IDIOMA"] = idioma_actual
    df["IDIOMA"] = df["IDIOMA"].ffill()
    df = df[df["IDIOMA"] == idioma_input].copy()
    df = df[df["CODIGO"].notna()]
    df = df[~df["CODIGO"].astype(str).str.upper().isin(idiomas_validos)]
    df = df[~df["CODIGO"].astype(str).str.upper().str.contains("CODIGO")]
    df["DOCENTE"] = df["DOCENTE"].ffill()

    # Nivel y Ciclo
    def extraer_nivel_y_ciclo(valor):
        valor = str(valor).strip().upper()
        if "REPASO" in valor:
            return "", "", "repaso"
        match = re.match(r"([BIA])(\d+)", valor)
        if match:
            nivel_map = {"B": "Básico", "I": "Intermedio", "A": "Avanzado"}
            return nivel_map.get(match.group(1), ""), match.group(2), None
        return "", "", None

    niveles = []
    ciclos = []
    overrides = []
    for i, valor in enumerate(df.get("CICLO", [])):
        try:
            result = extraer_nivel_y_ciclo(valor)
            if not isinstance(result, (list, tuple)) or len(result) != 3:
                result = ("", "", None)
        except Exception as e:
            result = ("", "", None)
        nivel, ciclo, override = result
        niveles.append(nivel)
        ciclos.append(ciclo)
        overrides.append(override)
    df["Nivel"] = niveles if niveles else None
    df["Ciclo"] = ciclos if ciclos else None
    df["_mod_override"] = overrides if overrides else None

    if "_mod_override" in df.columns and "MODALIDAD" in df.columns:
        df["MODALIDAD"] = df.apply(
            lambda row: row["_mod_override"] if pd.notna(row.get("_mod_override")) else row.get("MODALIDAD"),
            axis=1
        )
        df.drop(columns=["_mod_override"], inplace=True)

    # Días detectados
    def extraer_dias(texto):
        if pd.isna(texto): return []
        texto = texto.upper().replace(" Y ", ", ")
        dias_validos = ["LUNES", "MARTES", "MIÉRCOLES", "JUEVES", "VIERNES", "SÁBADOS", "DOMINGOS"]
        return [d for d in map(str.strip, texto.split(",")) if d in dias_validos]

    df["DÍAS DETECTADOS"] = df["DIAS"].apply(extraer_dias)

    # Inscritos y esperados
    def separar_inscritos(val):
        if pd.isna(val): return pd.Series([None, None])
        val = str(val).strip()
        if "/" in val:
            try:
                num, esperado = val.split("/")
                return pd.Series([int(num), int(esperado)])
            except:
                return pd.Series([None, None])
        elif val.isdigit():
            return pd.Series([int(val), None])
        return pd.Series([None, None])

    if "Nª inscritos" in df.columns:
        df[["N° Inscritos", "N° Esperado"]] = df["Nª inscritos"].apply(separar_inscritos)
    else:
        df["N° Inscritos"] = None
        df["N° Esperado"] = None

    # Horario detallado estructurado
    dia_a_codigo = {
        "LUNES": 0, "MARTES": 1, "MIÉRCOLES": 2,
        "JUEVES": 3, "VIERNES": 4, "SÁBADOS": 5, "DOMINGOS": 6
    }
    def parse_hora(hora_str):
        try:
            return datetime.strptime(hora_str.strip(), "%H:%M").time()
        except:
            return None

    def mapear_horarios_especial(dias, horas):
        if not isinstance(horas, str) or not dias:
            return {}
        bloques = [h.strip() for h in horas.split(",")]
        resultado = {}
        if len(bloques) == 2 and len(dias) >= 3:
            try:
                h1_inicio, h1_fin = map(parse_hora, bloques[0].split(" - "))
                h2_inicio, h2_fin = map(parse_hora, bloques[1].split(" - "))
                resultado[dia_a_codigo[dias[0]]] = (h1_inicio, h1_fin)
                for d in dias[1:]:
                    resultado[dia_a_codigo[d]] = (h2_inicio, h2_fin)
            except:
                return {}
        elif len(bloques) == 1:
            try:
                h_inicio, h_fin = map(parse_hora, bloques[0].split(" - "))
                for d in dias:
                    resultado[dia_a_codigo[d]] = (h_inicio, h_fin)
            except:
                return {}
        return resultado

    df["HORARIO DETALLADO"] = df.apply(lambda row: mapear_horarios_especial(row["DÍAS DETECTADOS"], row["HORAS"]), axis=1)

    # Fechas como date (con formato seguro)
    for col in ["F. Inicio", "F. Fin", "Parcial", "Final", "Subida de notas"]:
        df[col] = pd.to_datetime(df[col], format="%Y-%m-%d", errors='coerce').dt.date

    # Convertir columnas a enteros o nulo
    columnas_enteras = [
        "Ciclo", "N° Inscritos", "N° Esperado",
        "N° Aprobados", "N° Desaprobados", "N° No asistio (tiene 0)"
    ]
    for col in columnas_enteras:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

    # Columnas finales
    columnas_finales = [
        "CODIGO", "Nivel", "Ciclo", "MODALIDAD", "DOCENTE", "IDIOMA", "DÍAS DETECTADOS",
        "HORARIO DETALLADO", "F. Inicio", "F. Fin",
        "Parcial", "Final", "Subida de notas",
        "N° Inscritos", "N° Esperado", "N° Aprobados", "N° Desaprobados",
        "N° No asistio (tiene 0)", "Destalle del curso"
    ]
    return df[columnas_finales].reset_index(drop=True)


In [126]:
def nombre_corto_curso(codigo_curso, df):
    fila = df[df["CODIGO"] == codigo_curso]
    if fila.empty:
        return f"❌ Código {codigo_curso} no encontrado."
    fila = fila.iloc[0]
    docente = str(fila["DOCENTE"]).strip()
    idioma = IDIOMA_ABBR.get(str(fila["IDIOMA"]).upper(), str(fila["IDIOMA"])[:3].upper())
    nivel = NIVEL_ABBR.get(fila["Nivel"], "NA")
    ciclo = str(fila["Ciclo"]).zfill(2) if pd.notna(fila["Ciclo"]) else "00"
    modalidad = MODALIDAD_ABBR.get(str(fila["MODALIDAD"]).lower(), "X")
    dias_abbr = "".join(sorted([DIA_COD.get(dia, "?") for dia in fila["HORARIO DETALLADO"].keys()]))
    horas_unicas = sorted(set([
        "-".join(f"{h.hour:02d}-{h.minute:02d}" for h in v if isinstance(h, time))
        for v in fila["HORARIO DETALLADO"].values()
        if isinstance(v, tuple) and all(isinstance(h, time) for h in v)
    ]))
    horario_final = "-".join(horas_unicas)
    return f"{docente}-{idioma} {modalidad}{ciclo}({nivel})-{dias_abbr}-{horario_final}"

In [127]:
# Parámetros del caso de uso
EXCEL_PATH = "Carga_Horaria_2025.xlsx"
MES = "JUNIO 2025"
IDIOMA = "ITALIANO"

# Obtener DataFrame limpio
df_cursos = clean_df_mes_idioma(EXCEL_PATH, MES, IDIOMA)
display(df_cursos)

,CODIGO,Nivel,Ciclo,MODALIDAD,DOCENTE,IDIOMA,DÍAS DETECTADOS,HORARIO DETALLADO,F. Inicio,F. Fin,Parcial,Final,Subida de notas,N° Inscritos,N° Esperado,N° Aprobados,N° Desaprobados,N° No asistio (tiene 0),Destalle del curso
0,25066254,Básico,3,intensivo,PAVEL PARI,ITALIANO,"[DOMINGOS, LUNES, MIÉRCOLES]","{6: (08:00:00, 13:20:00), 0: (19:00:00, 22:30:...",2025-06-30,2025-07-13,NaT,2025-07-13,2025-07-13,15,<NA>,<NA>,<NA>,<NA>,NaN
1,25066264,Básico,4,intensivo,PAVEL PARI,ITALIANO,"[DOMINGOS, LUNES, MIÉRCOLES]","{6: (08:00:00, 13:20:00), 0: (19:00:00, 22:30:...",2025-07-14,2025-07-27,NaT,2025-07-27,2025-07-28,15,<NA>,<NA>,<NA>,<NA>,NaN
2,25066237,Básico,1,regular,PAVEL PARI,ITALIANO,"[MARTES, JUEVES]","{1: (20:00:00, 22:30:00), 3: (20:00:00, 22:30:...",2025-06-24,2025-07-22,2025-07-03,2025-07-22,2025-07-24,6,<NA>,<NA>,<NA>,<NA>,NaN
3,25066277,Básico,5,regular,PAVEL PARI,ITALIANO,"[MARTES, JUEVES]","{1: (20:00:00, 22:30:00), 3: (20:00:00, 22:30:...",2025-06-24,2025-07-22,2025-07-03,2025-07-22,2025-07-24,12,<NA>,<NA>,<NA>,<NA>,En curso
4,25066336,Intermedio,1,regular,WALDEMIR AYALA,ITALIANO,"[LUNES, MIÉRCOLES]","{0: (20:00:00, 22:30:00), 2: (20:00:00, 22:30:...",2025-06-23,2025-07-21,2025-07-02,2025-07-21,2025-07-23,6,<NA>,<NA>,<NA>,<NA>,NaN


In [128]:
def exportar_inscritos_formato_morado(
    codigo_curso,
    df_curso,
    feriados,
    plantilla_path="plantilla_lista_estudiantes.xlsx",
    carpeta_entrada="./",
    carpeta_salida="./"
):
    # 1. Nombre de archivo destino (usando tu función)
    nombre_salida = nombre_corto_curso(codigo_curso, df_curso) + ".xlsx"
    nombre_salida = nombre_salida.replace("/", "-")
    ruta_destino = str(Path(carpeta_salida) / nombre_salida)

    # 2. Copiar plantilla (mantiene formato)
    shutil.copy(plantilla_path, ruta_destino)

    # 3. Leer plantilla y archivo de inscritos
    wb = load_workbook(ruta_destino)
    ws = wb.active
    df_inscritos = pd.read_excel(f"{carpeta_entrada}/Inscritos_{codigo_curso}.xlsx")

    n_estudiantes = df_inscritos.shape[0]

    # 4. Copiar formato de la fila 2 (A-E) hacia abajo para cada estudiante
    for i in range(n_estudiantes):
        source_row = 2
        target_row = 2 + i
        for col in range(1, 6):  # columnas A-E
            cell_src = ws.cell(row=source_row, column=col)
            cell_tgt = ws.cell(row=target_row, column=col)
            cell_tgt._style = copy(cell_src._style)
            cell_tgt.font = copy(cell_src.font)
            cell_tgt.border = copy(cell_src.border)
            cell_tgt.fill = copy(cell_src.fill)
            cell_tgt.number_format = copy(cell_src.number_format)
            cell_tgt.protection = copy(cell_src.protection)
            cell_tgt.alignment = copy(cell_src.alignment)

    # 5. Llenar datos en las filas A3-E{n}
    for idx, row in enumerate(df_inscritos.itertuples(index=False), start=2):
        ws[f"A{idx}"] = idx - 1
        ws[f"B{idx}"] = row.CODIGO_CURSO
        ws[f"C{idx}"] = row.NOMBRES
        ws[f"D{idx}"] = row.CORREO
        ws[f"E{idx}"] = row.CELULAR

    # 6. Poner modalidad, nivel, ciclo en G2, fechas en H2 e I2
    fila_curso = df_curso[df_curso["CODIGO"] == codigo_curso].iloc[0]
    nivel = fila_curso["Nivel"]
    ciclo = str(fila_curso["Ciclo"]).zfill(2) if pd.notna(fila_curso["Ciclo"]) else ""
    modalidad_nivel_ciclo = f"{MODALIDAD_ABBR.get(str(fila_curso['MODALIDAD']).lower(), 'X')} {NIVEL_ABBR.get(nivel, 'X')}{ciclo}"
    ws["G2"] = modalidad_nivel_ciclo
    ws["H2"] = pd.to_datetime(fila_curso["F. Inicio"])
    ws["I2"] = pd.to_datetime(fila_curso["F. Fin"])
    ws["H2"].number_format = 'DD-MMM'
    ws["I2"].number_format = 'DD-MMM'

    # 7. Poner feriados debajo de "Feriados" en G4 para abajo
    ws["G4"] = "Feriados"
    for i, f in enumerate(feriados):
        ws.cell(row=5+i, column=7).value = f

    wb.save(ruta_destino)
    print("✅ Exportado:", ruta_destino)
    return ruta_destino

In [129]:
# df_curso = tu dataframe de cursos
codigo_curso = 25066254
feriados = ['2025-06-29']

feriados = ["2025-06-29"]
exportar_inscritos_formato_morado(
    codigo_curso,
    df_cursos,
    feriados,
    carpeta_entrada="./inscritos/",
    carpeta_salida="./output/"
)


✅ Exportado: output\PAVEL PARI-ITA INT03(B)-DLX-08-00-13-20-19-00-22-30.xlsx


'output\\PAVEL PARI-ITA INT03(B)-DLX-08-00-13-20-19-00-22-30.xlsx'